<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/StabilityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.1 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

import sympy
from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig
from scipy.integrate import solve_ivp
from scipy.optimize import root, minimize

In [16]:
Ac = 0.9
b = 0.29
Kc=0.3
qin_ss = 1.12
h_initial_ss = np.full(3, qin_ss/b)
h3sp_initial = h_initial_ss[-1]


def rhs(t, v, h3sp):
    h1, h2, h3 = v
    err = h3sp  - h3
    q12 = b*h1
    q23 = b*h2
    q3 = b*h3
    qin1 = qin_ss + Kc*err

    dh1  = (qin1 - q12)/Ac
    dh2 = (q12 - q23)/Ac
    dh3 = (q23 - q3 )/Ac
    return jnp.array([dh1, dh2, dh3])

In [21]:
tend=100
res=solve_ivp(lambda t, v: rhs(t, v, h3sp_initial+1.), (0,tend ), h_initial_ss, method='Radau', dense_output=True)
res

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.190e-01 ...  6.876e+01  1.000e+02]
        y: [[ 3.862e+00  3.901e+00 ...  4.370e+00  4.371e+00]
            [ 3.862e+00  3.863e+00 ...  4.370e+00  4.371e+00]
            [ 3.862e+00  3.862e+00 ...  4.371e+00  4.371e+00]]
      sol: <scipy.integrate._ivp.common.OdeSolution object at 0x79fcabab9990>
 t_events: None
 y_events: None
     nfev: 86
     njev: 1
      nlu: 22

In [22]:
tplot = np.linspace(0,tend,500)
fig=make_subplots()
# fig.add_scatter(x=t,y=h3_dev+h_initial_ss[2], name='linear')
fig.add_scatter(x=tplot, y= res.sol(tplot)[2], name='solve_ivp')
fig.update_layout(width=600, height=400, template='plotly_dark')

In [23]:
K=1/b
tau = Ac/b
Kc*K/(1+Kc*K)

0.5084745762711865

In [24]:
res.sol(tplot)[2][-1] - res.sol(tplot)[2][0]

0.5084768590754143

In [25]:
w = np.logspace(-2,2,1000)

In [26]:
RI=Kc*K/((tau*w*1j +1)**3)
R = np.real(RI)
I = np.imag(RI)

In [28]:
Gp =Kc*K/((tau*s + 1)**3)
t, y = sim(Gp, lambda t: np.sin(0.1*t), N=5000, dt=0.1)

In [29]:
fig3=make_subplots()
fig3.add_scatter(x=t, y= y)
fig3.update_layout(width=600, height=400, template='plotly_dark')

In [30]:
AR=np.abs(RI)

In [31]:
phase = np.unwrap(np.arctan2(I,R))*180/np.pi

In [32]:
fig2 = make_subplots(rows=2,cols=1)
fig2.add_scatter(x=w, y=AR, row=1, col=1)
fig2.update_xaxes(type="log", row=1, col=1)
fig2.update_yaxes(type='log', row=1,col=1)
fig2.add_scatter(x=w, y=phase, row=2, col=1)
fig2.update_xaxes(type="log", row=2, col=1)
fig2.update_layout(height=600, width=400, template='plotly_dark', showlegend=False,
                   yaxis2=dict(dtick=60))